In [1]:
import os

In [2]:
%pwd

'/Users/harendrakumar/Documents/Demand_forecast/research'

In [3]:
os.chdir("../")
%pwd

'/Users/harendrakumar/Documents/Demand_forecast'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
from Demand_Forecast.constants import *
from Demand_Forecast.utils.common import read_yaml, create_directories

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: str
    source_URL: str
    local_data_file: str
    unzip_dir: Path

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

In [8]:
import os
import urllib.request as requests
import zipfile
from Demand_Forecast import logger
from Demand_Forecast.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig)->None:
        self.config = config
    

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = requests.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )

            logger.info(f"{filename} downloaded! with the following info: {headers}")
        
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    

    def extract_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        
        

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_file()
except Exception as e:
    print(e)

[2023-10-16 13:32:46,775: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-16 13:32:46,776: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-16 13:32:46,777: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-16 13:32:46,778: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-16 13:32:48,427: INFO: 185851671: artifacts/data_ingestion/data.zip downloaded! with the following info: Connection: close
Content-Length: 1493
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3a3a5f968e05f78ca8f43fa36c36777d6fa59115ba5ca8d152cc336cca343aa2"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 692C:25C072:20346:2E0D8:652CEE27
Accept-Ranges: bytes
Date: Mon, 16 Oct 2023 08:02:48 GMT
Via: 1.1 varnish
X-Served-By: cache-del21729-DEL
